In [1]:
# Remove deprecationWarning that I can't fix
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [2]:

import re
import numpy as np
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy for lemmatization
import spacy


from pprint import pprint
from nltk.corpus import stopwords


# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# NLTK for removing stopwords, 
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#Apart from the known english stopwords, these are the pre-determined words that are common in the corpus
stop_words.extend(['tagum', 'mayor', 'series', 'enter','entered', 'behalf', 'known', 'otherwise', 
                  'code', 'sign', 'comprehensive', 'agreement', 'resolution', 'new', 'memorandum', 'city', 
                  'davao','ordinance', 'thereof', 'section', 'municipal', 
                  'municipality', 'government'])

In [4]:
# Import CSV Ordinance Dataset
df = pd.read_csv('ordinances/davao_ordinances.csv', header = 0)
print(df)
df.head()
# List all ordinance titles
# Convert to list
data = list(df.OT) 
print(data[:1])

            OR                                                 OT
0    000172-14  An Ordinance amending Section 42 of the 2005 R...
1    000177-14  An Ordinance placing underground all electrica...
2    000178-14  An Ordinance amending City Ordinance No. 0195-...
3    000173-14  An Ordinance regulating the use of some street...
4    000227-14  An ordinance for the creation of Trust Fund Ac...
..         ...                                                ...
414  000496-13  An Ordinance authorizing the City Mayor to ent...
415  000487-16  An Ordinance amending Section 13 Article XI of...
416  000348-07  Ordinance  Establishing the Gender-Sensitive C...
417  000349-07  ORDINANCE AMENDING DAVAO CITY ORDINANCE NO. 15...
418  000355-07  An  Ordinance  for  the  "Davao  Branding  Sys...

[419 rows x 2 columns]
['An Ordinance amending Section 42 of the 2005 Revenue Code of Davao City from "Time and Place of the Payment - The tax shall be due and payable in advance to the City Treasurer or his

In [5]:
# Remove qoutes
data = [re.sub("\'", "", sent) for sent in data]
data = [re.sub("\"", "", sent) for sent in data]

# Remove words ending with ING like amending, regulating
# Implemtation to be improved / Seems to improve topics
data = [re.sub('\S*ING\S*\s?', '', sent) for sent in data]
data = [re.sub('\S*ing\S*\s?', '', sent) for sent in data]


pprint(data[:5])

#preprocess using gensim simple_preprocess and tokenize into words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

pprint(data_words[:5])

['An Ordinance Section 42 of the 2005 Revenue Code of Davao City from Time and '
 'Place of the Payment - The tax shall be due and payable in advance to the '
 'City Treasurer or his duly authorized representative before the materials '
 'are extracted and shall be based on the volume applied for in the '
 'application for quarry concession to Tax shall be due and payable every '
 'quarter of the Calendar Year to the City Treasurer or his duly authorized '
 'representative after the materials are extracted based on the volume applied '
 'for in the application for quarry concession',
 'An Ordinance underground all electrical and telecommunication wires and '
 'cables within the vicinity of City Hall and the Sangguniang Panlungsod of '
 'the City of Davao',
 'An Ordinance City Ordinance No. 0195-14, Series of 2004, otherwise known as '
 'the Amended scheme of Davao City for a 90-day experimental period',
 'An Ordinance the use of some streets near the Mintal Public Market in '
 'Baranga

<>:7: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \S
<>:7: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \S
/var/folders/5w/v1hv7scs4gl14_30l_pjsrj00000gn/T/ipykernel_2229/1833599370.py:7: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*ING\S*\s?', '', sent) for sent in data]
/var/folders/5w/v1hv7scs4gl14_30l_pjsrj00000gn/T/ipykernel_2229/1833599370.py:8: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*ing\S*\s?', '', sent) for sent in data]


In [6]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
# Code snippets from Selva Prabhakaran | Topic Modeling with Gensim (Python)
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [7]:

data_words_nostops = remove_stopwords(data_words)
data_words_nostops = [x for x in data_words_nostops if "amending" not in x]

In [8]:
# Create Dictionary
id2word = corpora.Dictionary(data_words_nostops)

# Create Corpus
texts = data_words_nostops

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 2), (3, 2), (4, 2), (5, 1), (6, 2), (7, 2), (8, 2), (9, 1), (10, 2), (11, 2), (12, 2), (13, 1), (14, 1), (15, 2), (16, 1), (17, 2), (18, 1), (19, 3), (20, 2), (21, 1), (22, 2), (23, 2), (24, 1)]]


In [9]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('advance', 1),
  ('application', 2),
  ('applied', 2),
  ('authorized', 2),
  ('based', 2),
  ('calendar', 1),
  ('concession', 2),
  ('due', 2),
  ('duly', 2),
  ('every', 1),
  ('extracted', 2),
  ('materials', 2),
  ('payable', 2),
  ('payment', 1),
  ('place', 1),
  ('quarry', 2),
  ('quarter', 1),
  ('representative', 2),
  ('revenue', 1),
  ('shall', 3),
  ('tax', 2),
  ('time', 1),
  ('treasurer', 2),
  ('volume', 2),
  ('year', 1)]]

In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=36, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=30,
                                           alpha='auto',
                                           per_word_topics=True)

In [11]:
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(12, '0.001*"card" + 0.001*"issued" + 0.001*"acts" + 0.001*"careless" + 0.001*"insanitary" + 0.001*"refuse" + 0.001*"twenty" + 0.001*"id" + 0.001*"mindanao" + 0.001*"utilitiy"'), (7, '0.057*"panlungsod" + 0.057*"sangguniang" + 0.021*"underground" + 0.019*"vicinity" + 0.009*"wires" + 0.009*"hall" + 0.009*"cables" + 0.009*"electrical" + 0.000*"puvs" + 0.000*"flood"'), (27, '0.123*"anti" + 0.009*"smoke" + 0.000*"card" + 0.000*"issued" + 0.000*"insanitary" + 0.000*"refuse" + 0.000*"acts" + 0.000*"id" + 0.000*"mindanao" + 0.000*"utilitiy"'), (32, '0.107*"school" + 0.053*"elementary" + 0.025*"magsaysay" + 0.017*"ramon" + 0.017*"urban" + 0.015*"shelter" + 0.008*"daliao" + 0.008*"irr" + 0.008*"attached" + 0.008*"codeof"'), (9, '0.173*"buhangin" + 0.122*"road" + 0.073*"tigatto" + 0.022*"garcia" + 0.015*"barangays" + 0.006*"manuel" + 0.006*"mandug" + 0.000*"card" + 0.000*"id" + 0.000*"issued"'), (17, '0.282*"health" + 0.020*"sanitation" + 0.000*"refuse" + 0.000*"card" + 0.000*"id" + 0.000*"issu

In [12]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4465726353921931


In [13]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/Users/coriv/Developer/Ordinances_LDA/Ordinances_LDA/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
15     0.323316  0.126744       1        1  7.375773
16     0.175205 -0.344247       2        1  6.695237
14    -0.026521  0.004368       3        1  4.337852
5     -0.005736  0.019653       4        1  4.304055
26    -0.011746 -0.013564       5        1  4.265635
34    -0.032999  0.006176       6        1  4.013271
10     0.019552 -0.014175       7        1  3.821505
2      0.005310 -0.129272       8        1  3.691027
21     0.175159  0.102058       9        1  3.687154
1     -0.019003  0.005048      10        1  3.686595
18    -0.025216  0.003463      11        1  3.653021
3     -0.035248 -0.020369      12        1  3.468317
24     0.000968  0.017802      13        1  3.405276
22    -0.034662 -0.000493      14        1  3.352438
11    -0.017942  0.003101      15        1  3.319054
4      0.082615  0.065986      16        1  3.122037
31    -0.038209  0.007203      17        1  3.011380
6     -0.010730 -0.001274      18        1  2.982551
30    -0.032785 -0.046370      19        1  2.922427
28     0.028743  0.030420      20        1  2.909765
29    -0.022899  0.010739      21        1  2.859606
13    -0.035280  0.008104      22        1  2.546599
19    -0.023517 -0.003323      23        1  2.469200
25    -0.034553  0.011841      24        1  2.137360
0     -0.037403  0.008704      25        1  1.994958
23    -0.011795  0.035072      26        1  1.769579
33    -0.031713 -0.008809      27        1  1.724421
35    -0.039370  0.015509      28        1  1.158536
20    -0.038574  0.014894      29        1  1.119596
17    -0.035721  0.015313      30        1  0.928255
8     -0.039318 -0.005467      31        1  0.866438
9     -0.035535  0.015935      32        1  0.734829
32    -0.035821  0.015602      33        1  0.723330
7     -0.034638  0.015235      34        1  0.482814
27    -0.033529  0.014778      35        1  0.371307
12    -0.030407  0.013613      36        1  0.088805, topic_info=             Term       Freq      Total Category  logprob  loglift
39       barangay  78.000000  78.000000  Default  30.0000  30.0000
44         public  55.000000  55.000000  Default  29.0000  29.0000
157      purposes  59.000000  59.000000  Default  28.0000  28.0000
104          area  40.000000  40.000000  Default  27.0000  27.0000
220  construction  39.000000  39.000000  Default  26.0000  26.0000
..            ...        ...        ...      ...      ...      ...
25         cables   0.002576   1.251586  Topic36  -7.5756   0.8407
26     electrical   0.002576   1.251586  Topic36  -7.5756   0.8407
27           hall   0.002576   1.251586  Topic36  -7.5756   0.8407
28     panlungsod   0.002576   2.558013  Topic36  -7.5756   0.1258
29    sangguniang   0.002576   2.558013  Topic36  -7.5756   0.1258

[1519 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
533      16  0.577665     abrille
139      16  0.871014      accept
671       8  0.960232  accordance
48        6  0.642827     account
625       2  0.791728  accredited
...     ...       ...         ...
24       10  0.909468        year
24       28  0.060631        year
422      19  0.615178      yearly
515       6  0.694158       years
496       8  0.580475      zonees

[707 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 17, 15, 6, 27, 35, 11, 3, 22, 2, 19, 4, 25, 23, 12, 5, 32, 7, 31, 29, 30, 14, 20, 26, 1, 24, 34, 36, 21, 18, 9, 10, 33, 8, 28, 13])

/Users/coriv/Developer/Ordinances_LDA/Ordinances_LDA/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/coriv/Developer/Ordinances_LDA/Ordinances_LDA/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/coriv/Developer/Ordinances_LDA/Ordinances_LDA/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/coriv/Developer/Ordinances_LDA/Ordinances_LDA/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative us